<a href="https://colab.research.google.com/github/jai8004/Av-tuts/blob/main/Naive_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# store openapi key in oracle_api_key  in secrets colab
from google.colab import userdata
import os
os.apikey = userdata.get('nexusiqkey_F06752')

In [ ]:
# !pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.41
    Uninstalling langchain-core-0.3.41:
      Successfully uninstalled langchain-core-0.3.41


In [ ]:
# !pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.6 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = userdata.get('nexusiqkey_F06752')
llm = ChatOpenAI(model ="gpt-4o-mini")
llm.invoke("hi, What is Vaughan is saying about champions trophy").content

"As of my last update in October 2023, I don't have access to real-time information or statements from individuals. If you're referring to former England cricketer Michael Vaughan, he often shares his opinions on cricket events, including tournaments like the ICC Champions Trophy, through social media or other platforms. \n\nTo find Vaughn's latest thoughts on the Champions Trophy, I recommend checking his Twitter account, sports news websites, or cricket analysis platforms for the most current insights and commentary. If you need a summary of any historical context or past comments related to the Champions Trophy or Vaughan's views, feel free to ask!"

In [ ]:
llm.invoke("hi, which country won champion's tropee 2025?").content

'As of my last update in October 2023, I do not have information on the winner of the ICC Champions Trophy 2025, as the event had not yet occurred. For the most accurate and up-to-date information, please check a reliable sports news source or the official ICC website.'

In [ ]:
# prompt: implement RAg by using langchain and using in memory vector store

from google.colab import userdata
import os
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import InMemoryDocstore
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
# store openapi key in oracle_api_key  in secrets colab
os.environ["OPENAI_API_KEY"] = userdata.get('nexusiqkey_F06752')

llm = ChatOpenAI(model="gpt-4o-mini")

# Example documents (replace with your actual documents)
sample_doc = """
NEW DELHI: Former England captain Michael Vaughan has heaped praise on India’s extraordinary bench strength following their historic ICC Champions Trophy victory over New Zealand on Sunday.
Taking to X (formerly Twitter), Vaughan suggested that even a second-string Indian side could have reached the final.
“Jaiswal, Varma, Sharma, SKY, Pant, Reddy, Sundar, Chahal, Arshdeep, Bumrah & Bishnoi… This team would have made it to the final as well IMO. The strength in Indian cricket in white-ball is off the scale,” he wrote.
"""
# Create a document loader
loader = TextLoader(file_path='/content/sample_doc.txt') # Dummy filepath, content will be directly used
#  Write the sample doc to the dummy file
with open("/content/sample_doc.txt","w") as f:
  f.write(sample_doc)
documents = loader.load()
# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Create embeddings
embeddings = OpenAIEmbeddings()

# Create vector store
docsearch = InMemoryDocstore({d.page_content: d for d in docs})#InMemoryDocstore(embeddings.embed_query).from_documents(documents)

from langchain.retrievers import SVMRetriever
retriever = SVMRetriever.from_texts(
    texts=[d.page_content for d in docs],
    embeddings=embeddings,
    docstore=docsearch
)

# Create the chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Example queries
query1 = "hi, What is Vaughan is saying about champions trophy"
result1 = qa.run(query1)
print(f"Query 1 Result: {result1}")

query2 = "hi, which country won champion's tropee 2025?"
result2 = qa.run(query2)
print(f"Query 2 Result: {result2}")


<ipython-input-7-a33d77f2c087>:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = qa.run(query1)


Query 1 Result: Michael Vaughan is praising India's bench strength in cricket following their victory in the ICC Champions Trophy. He suggested that even a second-string Indian team, which includes players like Jaiswal, Varma, Sharma, and others, could have reached the final of the tournament. He highlighted the impressive depth of talent in Indian white-ball cricket.
Query 2 Result: I don't know.


In [ ]:
# prompt: implement RAg by using langchain and using in memory vector store

from google.colab import userdata
import os
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import InMemoryDocstore
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
# store openapi key in oracle_api_key  in secrets colab
os.environ["OPENAI_API_KEY"] = userdata.get('nexusiqkey_F06752')

llm = ChatOpenAI(model="gpt-4o-mini")

# Example documents (replace with your actual documents)
# sample_doc = """
# NEW DELHI: Former England captain Michael Vaughan has heaped praise on India’s extraordinary bench strength following their historic ICC Champions Trophy victory over New Zealand on Sunday.
# Taking to X (formerly Twitter), Vaughan suggested that even a second-string Indian side could have reached the final.
# “Jaiswal, Varma, Sharma, SKY, Pant, Reddy, Sundar, Chahal, Arshdeep, Bumrah & Bishnoi… This team would have made it to the final as well IMO. The strength in Indian cricket in white-ball is off the scale,” he wrote.
# """
# Create a document loader
loader = TextLoader(file_path='/content/Nayan_CV.txt') # Dummy filepath, content will be directly used
#  Write the sample doc to the dummy file
# with open("/content/sample_doc.txt","w") as f:
#   f.write(sample_doc)
documents = loader.load()
# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Create embeddings
embeddings = OpenAIEmbeddings()

# Create vector store
docsearch = InMemoryDocstore({d.page_content: d for d in docs})#InMemoryDocstore(embeddings.embed_query).from_documents(documents)

from langchain.retrievers import SVMRetriever
retriever = SVMRetriever.from_texts(
    texts=[d.page_content for d in docs],
    embeddings=embeddings,
    docstore=docsearch
)

# Create the chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Example queries
query1 = "hi, What is Nayan's Total experience?"
result1 = qa.run(query1)
print(f"Query 1 Result: {result1}")

query2 = "hi, What are Nayan's core skills??"
result2 = qa.run(query2)
print(f"Query 2 Result: {result2}")


Query 1 Result: Nayan Agrawal has a total of approximately 5 years of work experience. They worked as a Graduate Engineer Trainee from June 2018 to May 2019 and have been a Product Design Manager since June 2019, currently still in that role as of October 2023.
Query 2 Result: Nayan's core skills include:

- Machine Learning
- Deep Learning
- Natural Language Processing (NLP)
- Feature Engineering

Additionally, he is proficient in various tools such as Python, SQL, Power-BI, FastAPI, Flask, Django, Scikit-Learn, TensorFlow, OpenCV, and DialogFlow.


In [ ]:
# prompt: implement RAg by using langchain and using in memory vector store

from google.colab import userdata
import os
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import InMemoryDocstore
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import TextLoader, PyPDFLoader


# store openapi key in oracle_api_key  in secrets colab
os.environ["OPENAI_API_KEY"] = userdata.get('nexusiqkey_F06752')

llm = ChatOpenAI(model="gpt-4o-mini")

# Example documents (replace with your actual documents)
# sample_doc = """
# NEW DELHI: Former England captain Michael Vaughan has heaped praise on India’s extraordinary bench strength following their historic ICC Champions Trophy victory over New Zealand on Sunday.
# Taking to X (formerly Twitter), Vaughan suggested that even a second-string Indian side could have reached the final.
# “Jaiswal, Varma, Sharma, SKY, Pant, Reddy, Sundar, Chahal, Arshdeep, Bumrah & Bishnoi… This team would have made it to the final as well IMO. The strength in Indian cricket in white-ball is off the scale,” he wrote.
# """
# Create a document loader
loader = PyPDFLoader(file_path='Nayan Agrawal CV.pdf')
# loader = TextLoader(file_path='/content/Nayan_CV.txt') # Dummy filepath, content will be directly used
#  Write the sample doc to the dummy file
# with open("/content/sample_doc.txt","w") as f:
#   f.write(sample_doc)
documents = loader.load()
# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Create embeddings
embeddings = OpenAIEmbeddings()

# Create vector store
docsearch = InMemoryDocstore({d.page_content: d for d in docs})#InMemoryDocstore(embeddings.embed_query).from_documents(documents)

from langchain.retrievers import SVMRetriever
retriever = SVMRetriever.from_texts(
    texts=[d.page_content for d in docs],
    embeddings=embeddings,
    docstore=docsearch
)

# Create the chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Example queries
query1 = "hi, What is Nayan's Total experience?"
result1 = qa.run(query1)
print(f"Query 1 Result: {result1}")

query2 = "hi, What are Nayan's core skills??"
result2 = qa.run(query2)
print(f"Query 2 Result: {result2}")


<ipython-input-3-3f62eb9107e0>:38: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
<ipython-input-3-3f62eb9107e0>:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = qa.run(query1)


Query 1 Result: Nayan has a total work experience of about 1 year and 4 months. They worked as a Graduate Engineer Trainee from June 2018 to May 2019 and have been a Product Design Manager since June 2019, continuing in that role to the present.
Query 2 Result: Nayan's core skills include:

- Machine Learning
- Deep Learning
- Natural Language Processing (NLP)
- Feature Engineering

He also has experience with various tools such as Python, SQL, Power-BI, FastAPI, Flask, Django, Scikit-Learn, TensorFlow, OpenCV, and DialogFlow.
